In [136]:
import notebookimport

In [137]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")
sp = __import__("Charts - Stage Progress")


In [138]:
#need to get db builder -
#eg http://localhost:8888/notebooks/Documents/code/github%20forks/WRC_sketches/doodles/WRC%20API%202018%20Base.ipynb

In [139]:
import os
import sqlite3
import pandas as pd

#dbname='wrc18.db'
dbname='france18.db'
conn = sqlite3.connect(dbname)

In [140]:
rally='France'

In [141]:
'''year=2018
rc='RC2'
ss='SS4'


typ='stage_times_stage' #stage_times_stage stage_times_overall
typ='stage_times_overall'''

"year=2018\nrc='RC2'\nss='SS4'\n\n\ntyp='stage_times_stage' #stage_times_stage stage_times_overall\ntyp='stage_times_overall"

In [142]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')
stagerank_overall = sr.getEnrichedStageRank(conn,rally, typ='overall')

stagerank_stage = sr.getEnrichedStageRank(conn,rally, typ='stage')

In [143]:
basepath = 'report'
imgdir = 'images'
imgdirfull = '{}/{}'.format(basepath, imgdir)


In [144]:
sections = stageresults.groupby('section')

#Generate list from group: stageresults.groupby('section')['code'].apply(list)

#Generate group then iterate through groups
for key in sections.groups.keys():
    print(key, sections.get_group(key)['code'].tolist())


Section 1 ['SS1', 'SS2']
Section 2 ['SS3', 'SS4']
Section 3 ['SS5', 'SS6']
Section 4 ['SS7']
Section 5 ['SS8', 'SS9']
Section 6 ['SS10']
Section 7 ['SS11']
Section 8 ['SS12']


In [145]:
sectionREADME_base = '''\n### {section} Report
'''

## Section Reports

Can these be limited to e.g. just RC1?

In [146]:
q='''
    SELECT itc.code, itc.status
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND itc.code LIKE "SS%" 
    '''.format(rally=rally)
stage_status=pd.read_sql(q,conn).set_index('code').to_dict(orient='index')
stage_status

{'SS1': {'status': 'Interrupted'},
 'SS10': {'status': 'ToRun'},
 'SS11': {'status': 'ToRun'},
 'SS12': {'status': 'ToRun'},
 'SS2': {'status': 'Completed'},
 'SS3': {'status': 'Interrupted'},
 'SS4': {'status': 'Completed'},
 'SS5': {'status': 'ToRun'},
 'SS6': {'status': 'ToRun'},
 'SS7': {'status': 'ToRun'},
 'SS8': {'status': 'ToRun'},
 'SS9': {'status': 'ToRun'}}

In [147]:
%%capture
if not os.path.exists(imgdirfull):
    os.makedirs(imgdirfull)

def sectionRankingChart(data, fn,sectionREADME, key, sectionStages, deltalabels=True, elaborate=True):
    data = sr.stageOverallEnrichers(data)
    ss = data[data['code'].isin(sectionStages)].copy()
    ss = sr.stageLegEnrichers(ss)
    if ss.empty: return sectionREADME
    fn='{}/{}'.format(imgdirfull, fn_core)
    fig, ax = sr.plotStageProgressionChart( ss, linecolor='lightgrey' , deltalabels=deltalabels, elaborate=elaborate,
                                        progress=True, stageAnnotate=False, filename=fn, title = title); 
    if fig is not None:
        fn='{}/{}'.format(imgdir, fn_core)
        sectionREADME = '''{s}{key}\n\n![]({img})'''.format(s=sectionREADME, key=key,img=fn)
    return sectionREADME

for key in sections.groups.keys():
    #path = '{}/{}'.format(basepath,key)
    #if not os.path.exists(path):
    #    os.makedirs(path)
    sectionStages = sections.get_group(key)['code'].tolist()
    #Check section stages - if TORUN, then break
    for sectionStage in sectionStages:
        if stage_status[sectionStage]['status']=='ToRun': break
        
    sectionREADME = sectionREADME_base.format(section=key)
        
    #Stage rank chart
    fn_core='spchart_stage_{}.png'.format( key)
    title = '{} - Stage Ranking'.format(key)
    
    sectionREADME = sectionRankingChart(stagerank_stage, title, sectionREADME, key, sectionStages, deltalabels=False, elaborate=False)
    
    #Overall rally ranking chart
    fn_core='spchart_overall_{}.png'.format( key)
    title = '{} - Overall Rally Ranking Evolution'.format(key)
    sectionREADME = sectionRankingChart(stagerank_overall, title, sectionREADME, key, sectionStages, deltalabels=True, elaborate=True )
    
    with open('{bp}/{key}_report.md'.format(bp=basepath,key=key), 'a') as out_file:
        out_file.write(sectionREADME)


### Stage progression chart

In [148]:
%%capture
fn='{}/spchart_full.png'.format(imgdirfull)
fig, ax = sr.plotStageProgressionChart( stagerank_overall, linecolor='lightgrey' , deltalabels=True, 
                                        progress=True, stageAnnotate=False, filename=fn);

README='''\n![]({})'''.format('{}/spchart_full.png'.format(imgdir))
with open('{}/README.md'.format(basepath), 'a') as out_file:
    out_file.write(README)

In [149]:
!ls report

README.md           SS18_report.md      SS9_report.md       Section 8_report.md
SS10_report.md      SS19_report.md      SUMMARY.md          entries_RC1.md
SS11_report.md      SS20_report.md      Section 1_report.md entries_RC2.md
SS12_report.md      SS21_report.md      Section 2_report.md entries_RC3.md
SS13_report.md      SS22_report.md      Section 3_report.md entries_RC4.md
SS14_report.md      SS5_report.md       Section 4_report.md entries_RGT.md
SS15_report.md      SS6_report.md       Section 5_report.md images
SS16_report.md      SS7_report.md       Section 6_report.md
SS17_report.md      SS8_report.md       Section 7_report.md


## Stage Reports

In [150]:
overallleader = stagerank_overall[stagerank_overall['position']==1].set_index('code')[['driver.code']].to_dict(orient='dict')['driver.code']
overallleader

{'SS1': 'OGI', 'SS2': 'OGI', 'SS3': 'OGI', 'SS4': 'OGI'}

In [151]:
rc='RC1'
year=2018
ss='SS4'

In [152]:
import matplotlib.pyplot as plt
import unidecode

links=[]
for ss in stageresults['code']:
    #Check section stages - if TORUN, then break
    if stage_status[ss]['status']=='ToRun': break

    print(ss)
    #Should we avoid SS1?
    #SS1 stage bars are borked - disable for now?
    #if ss=='SS1': continue
    currprevstagerank = sp.getCurrPrevStageRank(conn, rally, rc, ss)
    splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
    # TO DO - SS1 stage bars are borked - disable for now?
    #Need to sort them acc. to driver who ends up first at SS1 end?
    if ss=='SS1': splitdurations=None
    with open('{}/{}_report.md'.format(basepath, ss), 'w') as f:
        #for dc in currprevstagerank['driver.code'].unique():
        dc = overallleader[ss]
        fig, ax = sp.rebaseStageProgressBar(currprevstagerank,dc, splitdurations=splitdurations,
                                            neg='lightgreen',pos='pink',
                                 title='Stage Progress Chart - {rally}, {year} - SS{ss}'.format(year=year,
                                                                                                rally=rally,
                                                                                                ss=str(ss).replace('SS','')));
        img='{}/stage_report_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
        fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
        plt.close(fig)  
        f.write('# Stage Progress Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                              dc=dc,
                                                                              ss=str(ss).replace('SS','')))
        f.write('![]({})\n'.format(img))

SS1
SS2
SS3
SS4


In [153]:
ssd.dbGetSplits(conn,rally,'SS10',rc='RC1').empty
#ssd.getSplitDurationsFromSplits(conn,rally,'SS9',rc)

True

In [154]:
!pwd

/Users/ajh59/Documents/code/github forks/WRC_sketches/doodles
